# Min-Max Games

## Rock Paper Scissors
### Moves
- Player X chooses Rock $x = (1,0,0)$, Paper $x = (0,1,0)$, or Scissors $x = (0,0,1)$
- Player Y does the same with $y$
- $x,y \in \mathbf{R}^3$

### Outcome
- Rock beats Scissors
- Scissors beats Paper
- Paper beats Rock
- Players **gain** $1$ point for a win
- Players **lose** $1$ point for a loss
- 0 points for ties

Consider $A$, the **gain matrix** for Player X, given by
$$
A = \begin{bmatrix}
 0 & -1 & +1\\
+1 &  0 & -1\\
-1 & +1 &  0
\end{bmatrix}.
$$

(Note that $A^T = -A$.)

That is $A_{ij}$ is the **gain** (positive or negative) of Player X in the event that

- X played $i \in \lbrace 1,2,3\rbrace$ (Rock, Paper, Scissors)
- Y played $j \in \lbrace 1,2,3\rbrace$ (Rock, Paper, Scissors)

More explicitly, if Player X chooses
$$
x \in \lbrace (1,0,0), (0,1,0), (0,0,1) \rbrace
$$
and Player Y chooses
$$
y \in \lbrace (1,0,0), (0,1,0), (0,0,1) \rbrace
$$
then the **gain** for Player X is
$$
x^T A y.
$$

Since one player's loss is the other's gain, Rock/Paper/Scissors is a **zero-sum game**, so the **gain** for Player Y is
$$
- x^T A y = x^T A^T y = (x^T A^T y)^T = y^T A x.
$$
We see that the game is symmetric with respect to the players.

### X's Strategy
#### Deterministic Strategies

Assuming Y goes first and X goes second, X's strategy is easy.

Given $y \in \lbrace (1,0,0), (0,1,0), (0,0,1) \rbrace$, X solves the problem

$$
\begin{array}{ll}
\mbox{maximize} & x^TAy \\
\mbox{subject to} & x \in \lbrace (1,0,0), (0,1,0), (0,0,1) \rbrace,
\end{array}
$$
which just amounts to choosing the largest element of $Ay$.

For example, if Y chooses Rock then $y = (1,0,0)$ and

$$Ay = (0,+1,-1).$$

Thus, X chooses Paper, $x = (0,1,0)$.

#### Probabilistic Strategies
Since X can easily defeat any deterministic choice by Y, Y decides to switch to a **randomized** strategy.

Y chooses a probability distribution $y \in \mathbf{R}^3$ such that
$$
\begin{align}
\mathbf{1}^T y = 1\\
y \geq 0.
\end{align}
$$

X does not know Y's choice of Rock, Paper, or Scissors ahead of time, but is aware of Y's **strategy** $y$.

X's best strategy is still easy. The **expected gain** of for any choice of X, knowing $y$ is
$$
Ay.
$$

X can still just choose the largest element of $Ay$.

Thus, X again solves the problem
$$
\begin{array}{ll}
\mbox{maximize} & x^TAy \\
\mbox{subject to} & x \in \lbrace (1,0,0), (0,1,0), (0,0,1) \rbrace,
\end{array}
$$

### Y's Strategy
Knowing that X will chose the best counter-strategy, Y wants to choose a strategy that will minimize her loss (equivalently, X's gain). Thus, Y solves the problem

$$
\begin{array}{ll}
\mbox{minimize} & \max_i(Ay) \\
\mbox{subject to} & \mathbf{1}^T y = 1\\
& y \geq 0
\end{array}
$$


In [21]:
from cvxpy import *
import numpy as np

A = np.array([[0,-1,+1],[+1,0,-1],[-1,+1,0]])
A.T == -A

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]], dtype=bool)

In [24]:
y = Variable(3)

obj = Minimize(max_entries(A*y))
constr = [sum_entries(y) == 1, y >= 0]
prob = Problem(obj,constr)
prob.solve()
y = np.array(y.value).flatten()
y

array([ 0.33333333,  0.33333333,  0.33333333])

- Y's best strategy is to distribute the probability evenly: $y = (1/3,1/3,1/3)$.
- Knowing Y's strategy, X's response doesn't matter, since X expects to come out even (0 gain) no matter what choice he makes:

In [25]:
A.dot(y)

array([  5.55111512e-17,  -2.77555756e-16,   2.22044605e-16])

### Constrained Y
- What if Y is constrained to play Rock 50% of the time?
- i.e., flip a coin; if heads, Y must play Rock; otherwise free to choose
- What is Y's best randomized strategy?

$$
\begin{array}{ll}
\mbox{minimize} & \max_i(Ay) \\
\mbox{subject to} & \mathbf{1}^T y = 1\\
& y \geq 0\\
& y_1 \geq 1/2
\end{array}
$$

In [26]:
y = Variable(3)

obj = Minimize(max_entries(A*y))
constr = [sum_entries(y) == 1, y >= 0, y[0] >= .5]
prob = Problem(obj,constr)
prob.solve()
y = np.array(y.value).flatten()
y

array([ 0.5       ,  0.16666667,  0.33333333])

### Interpretation
- $y^\star = (1/2, 1/6, 1/3)$
- Y plays Rock as little as possible: 50%
- X will play more Paper to exploit
- Y plays Scissors more often, 33.3%, to compensate

### Repeated Play
If playing many games, players can learn the other's strategy, and change theirs to exploit

Example:
- Y plays optimal strategy above
- X plays Paper 100% of the time to exploit Y's 50% Rock
- Y could **update** her strategy to $y = (1/2, 0, 1/2)$ to make the game even
- What is X's best strategy if Y can adjust?


### X's Strategy
- Assume that Y has some idea of X's randomized strategy, but is still constrained to play Rock 50% of the time
- Imagine X chooses $x$ first
- $x$ is revealed to Y
- Y chooses best reponse under her constraints
- Y computes $b = x^T A$
- Y solves 

$$
\begin{array}{ll}
g(x) = &\mbox{minimize} & (x^T A) y \\
&\mbox{subject to} & \mathbf{1}^T y = 1\\
&& y \geq 0\\
&& y_1 \geq 1/2
\end{array}
$$

- Then X solves
$$
\begin{array}{ll}
\mbox{maximize} & g(x) \\
\mbox{subject to} & \mathbf{1}^T x = 1\\
& x \geq 0
\end{array}
$$
- But how to figure out function $g(x)$?
- Normally use the topic of **duality**, but we won't here
- We simplified the inner optimization problem earlier
- Was easy to see that we just chose the optimal element
- Little more complicated in this case, but we can still do it pretty easily

### Y's Inner Strategy
- Y's choices can be boiled down to three:
    - $z_1 = (1, 0, 0)$
    - $z_2 = (1/2, 1/2, 0)$
    - $z_3 = (1/2, 0, 1/2)$
- Y chooses $y$ so that $x^T A y$ is smallest
- minimum over a finite set of functions


$$
\begin{array}{ll}
\mbox{maximize} & \min(x^T A z_1, x^T A z_2, x^T A z_3) \\
\mbox{subject to} & \mathbf{1}^T x = 1\\
& x \geq 0
\end{array}
$$

In [30]:
x = Variable(3)

y1 = np.array([1,0,0])
y2 = np.array([.5,.5,0])
y3 = np.array([.5,0,.5])

obj = Maximize(min_elemwise(x.T*A*y1,x.T*A*y2,x.T*A*y3))
constr = [sum_entries(x) == 1, x >= 0]
prob = Problem(obj,constr)
prob.solve()
x = np.array(x.value).flatten()
x

array([  3.33333333e-01,   6.66666667e-01,  -2.42218924e-10])

### X's Optimal Strategy
- $x^\star = (1/3, 2/3, 0)$
- Along with $y^\star = (1/2, 1/6, 1/3)$, gives the **equilibrium** strategies for the problem
- No player can improve their outcome by adjusting their strategy

### Outcome
- X has an advantage because Y is constrained to play Rock 50% of the time
- For each game, X's expected **gain** is $(x^\star)^T Ay^\star = 1/6$

In [35]:
x.T.dot(A).dot(y)

0.16666666679462955